In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("mbd-prj").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/05 10:50:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from pathlib import Path
lymsys_folder = Path("/Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/Big Data/Project/data/WildChat-1M-Full/WildChat-1M-Full/")
wildchat_folder = Path("/Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/Big Data/Project/data/WildChat-1M-Full/WildChat-1M-Full/")

In [9]:
# test reading the dataset from a folder of parquet files
lymsys_df = spark.read.parquet(str(lymsys_folder))
lymsys_df.show(5)
wildchat_df = spark.read.parquet(str(wildchat_folder))
wildchat_df.show(5)


+--------------------+------------------+-------------------+---------------------------------+----+--------+--------------------+--------------------+-----+--------+-------------+---------------+--------------------+--------------------+
|   conversation_hash|             model|          timestamp|                     conversation|turn|language|   openai_moderation| detoxify_moderation|toxic|redacted|        state|        country|           hashed_ip|              header|
+--------------------+------------------+-------------------+---------------------------------+----+--------+--------------------+--------------------+-----+--------+-------------+---------------+--------------------+--------------------+
|f01a345e668b978b6...|gpt-3.5-turbo-0613|2023-09-13 04:14:27|             [{(In the clubroo...|   1| English|[{{false, false, ...|[{2.1957975695841...|false|   false|     Michigan|  United States|f96c8515aa663db23...|{en-US,en;q=0.9, ...|
|b9af1406a2f1593d7...|gpt-3.5-turbo-0613|202